In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams

from keras.models import Sequential
from keras.layers import LSTM , Dropout , Dense

from sklearn.preprocessing import MinMaxScaler

rcParams['figure.figsize'] = 20 , 10
%matplotlib inline

In [ ]:
df = pd.read_csv('./Data/NSE-Tata.csv')

In [ ]:
df.head()

In [ ]:
#Analyze the closing prices from dataframe:
df['Date'] = pd.to_datetime(df.Date , format='%Y-%m-%d')
df.index = df['Date']

plt.figure(figsize=(16,8))
plt.plot(df.Close , label='Close Price History')

In [ ]:
df.head()

In [ ]:
# Sort the dataset on date time and filter “Date” and “Close” columns:
data = df.sort_index(ascending=True , axis=0)
new_data = pd.DataFrame(index=range(0,len(df)) , columns=['Date' , 'Close'])

for i in range(0,len(df)) :
    new_data['Date'][i] = data['Date'][i]
    new_data['Close'][i] = data['Close'][i]

In [ ]:
# Normalize the dataset
final_data = new_data.values
train_data = final_data[0:987,:]
valid_data = final_data[987:,:]

new_data.index = new_data.Date
new_data.drop('Date' , axis=1 , inplace=True)

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(new_data)

x_train , y_train = [] , []

for i in range(60 , len(train_data)) :
    x_train.append(scaled_data[i-60:i , 0])
    y_train.append(scaled_data[i , 0])
    
x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = np.reshape(x_train , (x_train.shape[0] , x_train.shape[1] , 1))

In [ ]:
# Build and train the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(units=50 , return_sequences=True , input_shape=(x_train.shape[1] , 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

input_data = new_data[len(new_data) - len(valid_data) - 60:].values
input_data = input_data.reshape(-1 , 1)
input_data = scaler.transform(input_data)

lstm_model.compile(loss='mean_squared_error' , optimizer='adam')
lstm_model.fit(x_train , y_train , epochs=1 , batch_size=1 , verbose=2)

In [ ]:
# Take a sample of a dataset to make stock price predictions using the model:
x_test = []
for i in range(60 , input_data.shape[0]) :
    x_test.append(input_data[i-60:i , 0])

In [ ]:
x_test = np.array(x_test)
x_test = np.reshape(x_test , (x_test.shape[0] , x_test.shape[1] , 1))

In [ ]:
predicted_prices = lstm_model.predict(x_test)

In [ ]:
predicted_prices = scaler.inverse_transform(predicted_prices)

In [ ]:
# Visualize the predicted stock costs with actual stock costs:
train_data = new_data[:987]
valid_data = new_data[987:]
valid_data['Prediction'] = predicted_prices
plt.figure(figsize=(16,8))
plt.plot(train_data['Close'])
plt.plot(valid_data[['Close',"Prediction"]])

In [ ]:
lstm_model.save("saved_model.h5")